# Command Line Interfaces (CLIs)

We an use TRL to fine-tune our language model with **Supervised Fine-Tuning (SFT)** or **Direct Policy Optimization (DPO)** or even chat with our model using the TRL CLIs.

**Training commands**
* `trl dpo`: fine-tune an LLM with DPO
* `trl grpo`: fine-tune an LLM with GRPO
* `trl kto`: fine-tune an LLM with KTO
* `trl sft`: fine-tune an LLM with SFT

**Other commands**
* `trl chat`: quickly spin up an LLM fine-tuned for chatting
* `trl env`: get the system information

## Fine-tuning with the CLI

We need to pick up a language model for text generation and a relevant dataset from HuggingFace Hub.

Before using the `sft` or `dpo` commands, we need to run:
```bash
accelerate config
```
and pick up the right configuration for our training setup (single / multi-GPU, DeepSpeed, etc.). Make sure to complete all steps of `accelerate config` before running any CLI command.

We also recommend passing a YAML config file to configure our training protocol. Below is a simple example of a YAML file that we can use for training our models with `trl sft` command:
```yaml
model_name_or_path: Qwen/Qwen2.5-0.5B
dataset_name: stanfordnlp/imdb
report_to: none
learning_rate: 0.0001
lr_scheduler_type: cosine
```
Save this file as `config.yaml` and we can get started immediately. We can also overwrite the arguments from the config file by explicitly passing them to the CLI, e.g., from the root folder:
```bash
trl sft --config path/to/configs/config.yaml --output_dir test-trl-cli --lr_scheduler_type cosine_with_restarts
```
which will force to use `cosine_with_restarts` for `lr_scheduler_type`.

### Supported Arguments

We can also set arguments from `transformers.TrainingArguments` for loading our model. This is all supported from the `trl.ModelConfig`.

### Supervised Fine-Tuning (SFT)

Follow the basic instructions above
```bash
trl stf --model_name_or_path facebook/opt-125m --dataset_name stanfordnlp/imdb --output_dir opt-sft-imdb
```
The SFT CLI is based on the `trl/scripts/sft.py` script.

### Direct Policy Optimization (DPO)

To use the DPO CLI, we need to have a dataset in the TRL format such as
* TRL's [Anthropic HH dataset](https://huggingface.co/datasets/trl-internal-testing/hh-rlhf-helpful-base-trl-style)
* TRL's [OpenAI TLDR summarization dataset](https://huggingface.co/datasets/trl-internal-testing/tldr-preference-trl-style)

These datasets always have at least three columns `prompt`, `chosen`, and `rejected`:
* `prompt` is a list of strings
* `chosen` is the chosen response in chat format
* `rejected` is the rejected response in chat format

Follow the basic instruction
```bash
trl dpo --model_name_or_path facebook/opt-125m --output_dir trl-hh-rlhf --dataset_name trl-internal-testing/hh-rlhf-helpful-base-trl-style
```
The DPO CLI is based on the `trl/scripts/dpo.py` script.

## Chat interface

The chat CLI lets us quickly load the model and talk to it by running
```bash
trl chat --model_name_or_path Qwen/Qwen1.5-0.5B-Chat
```

## Get the system information

We can get the system information by running
```bash
trl env
```
This will print out the system information including the GPU information, the CUDA version, the PyTorch version, the transformers version, and the TRL version, and any optional dependencies that are installed.

# Training Customization

## Train on multiple GPUs / nodes

The trainers in TRL use HuggingFace Accelerate to enalbe distributed training across multiple GPUs or nodes. To do so, we first need to create an Accelerate config file by running
```bash
accelerate config
```
and answering the questions according to our multi-gpu / multi-node setup. Then we can launch distributed training by running
```bash
accelerate launch our_script.py
```
We also provide config files in the [examples folder](https://github.com/huggingface/trl/tree/main/examples/accelerate_configs) that can be used as templates:
```bash
accelerate launch --config_file=examples/accelerate_configs/multi_gpu.yaml --num_processes {NUM_GPUS} path_to_script.py --all_arguments_of_the_script
```

### Distrubted training with DeepSpeed

All of the trainers in TRL can be run on multiple GPUs together with DeepSpeed ZeRO-{1,2,3} for efficient sharding of the optimizer states, gradients, and model weights:
```bash
accelerate launch --config_file=examples/accelerate_configs/deepspeed_zero{1,2,3}.yaml --num_processes {NUM_GPUS} path_to_your_script.py --all_arguments_of_the_script
```
For ZeRO-3, a small tweak is needed to intialize our reward model on the correct device viad the `zero3_init_context_manager()` context manager. This is needed to avoid DeepSpeed hanging after a fixed number of training steps.

## Use different optimizers and schedulers

By default, the `DPOTrainer` creates a `torch.optim.AdamW` optimizer. We can create and define a different optimizer and pass it to `DPOTrainer` as follows

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch import optim
from trl import DPOConfig, DPOTrainer

model_id = 'Qwen/Qwen2.5-0.5B-Instruct'
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
dataset = load_dataset('trl-lib/ultrafeedback_binarized', split='train')
training_args = DPOConfig(output_dir='Qwen2.5-0.5B-DPO')

optimizer = optim.SGD(model.parameters(), lr=training_args.learning_rate)

trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    optimizers=(optimizer, None), # no lr scheduler assigned
)
trainer.train()

### Add a learning rate scheduler

We can also play with our training by adding learning rate schedulers.

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch import optim
from trl import DPOConfig, DPOTrainer

model_id = 'Qwen/Qwen2.5-0.5B-Instruct'
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
dataset = load_dataset('trl-lib/ultrafeedback_binarized', split='train')
training_args = DPOConfig(output_dir='Qwen2.5-0.5B-DPO')

optimizer = optim.AdamW(model.parameters(), lr=training_args.learning_rate)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    optimizers=(optimizer, lr_scheduler),
)
trainer.train()

## Memory efficient fine-tuning by sharing layers

Another tool we can use for more memory efficient fine-tuning is to share layers between the reference model and the model we want to train.

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import create_reference_model, DPOConfig, DPOTrainer

model_id = 'Qwen/Qwen2.5-0.5B-Instruct'
model = AutoModelForCausalLM.from_pretrained(model_id)
ref_model = create_reference_model(model, num_shared_layers=6)
tokenizer = AutoTokenizer.from_pretrained(model_id)
dataset = load_dataset('trl-lib/ultrafeedback_binarized', split='train[:1%]')
training_args = DPOConfig(output_dir='Qwen2.5-0.5B-DPO')

trainer = DPOTrainer(
    model=model,
    ref_model=ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)
trainer.train()

## Pass 8-bit reference models

Since `trl` supports all keyword arguments when loading a model from `transformers` using `from_pretrained`, we can also leverage `load_in_8bit` from `transformers` for more memory efficient fine-tuning.

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import DPOConfig, DPOTrainer

model_id = 'Qwen/Qwen2.5-0.5B-Instruct'
model = AutoModelForCausalLM.from_pretrained(model_id)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
ref_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
dataset = load_dataset('trl-lib/ultrafeedback_binarized', split='train')
training_args = DPOConfig(output_dir='Qwen2.5-0.5B-DPO')

trainer = DPOTrainer(
    model=model,
    ref_model=ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)
trainer.train()

## Use the CUDA cache optimizer

When training large models, we should better handle the CUDA cache by iteratively clearing it. To do so, we simply pass `optimize_cuda_cache=True` to `DPOConfig`:

In [ ]:
training_args = DPOConfig(
    output_dir="Qwen2.5-0.5B-DPO",
    optimize_cuda_cache=True,
)

# Reducing Memory Usage

not finished yet. continue watching on HuggingFace

# Speeding Up Training

not finished yet. continue watching on HuggingFace

# Using model after training

## Load and Generate

If we have fine-tuned a model fully, meaning without the use of PEFT, we can simply load it like any other language model in `transformers`. The value head that was trained during the PPO training is no longer needed and it we load the model with the original transformer class it will be ignored.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name_or_path = "kashif/stack-llama-2" # path/to/our/model/or/name/on/hub
device = 'cpu'

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
inputs = tokenizer.encode(
    'This movie was really',
    return_tensors='pt'
).to(device)
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

Alternatively, we can also use the pipeline

In [ ]:
from transformers import pipeline

model_name_or_path = 'kashif/stack-llama-2'
pipe = pipeline('text-generation', model=model_name_or_path)

In [ ]:
print(pipe('This movie was really')[0]['generated_text'])

## Use PEFT Adapters

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = 'kashif/stack-llama-2'
adapter_model_name = 'path/to/my/adapter'

model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(model, adapter_model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [ ]:
inputs = tokenizer.encode(
    'This movie was really',
    return_tensors='pt'
).to(device)
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

We can also merge the adapters into the base model so we can use the model like a normal transformers model:

In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = 'kashif/stack-llama-2'
adapter_model_name = 'path/to/my/adapter'

model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(model, adapter_model_name)

model = model.merge_and_unload()
model.save_pretrained('merged_model')